In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv('info.csv')

In [2]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(data['center'],data['steering'],test_size =0.2 ,random_state=25)
print("train length{},val length{}".format(len(X_train),len(X_test)))

train length1519,val length380


In [21]:
from imgaug import augmenters as iaa
import numpy as np

def zoom_img(img):
    # get a resizer for an image
    zoom = iaa.Affine(scale=(1.0,1.3))
    return zoom.augment_image(img)
def trans_img(img):
    # get an image translator
    trans = iaa.Affine(translate_percent={'x':(-0.1,0.1),'y':(-0.1,0.1)})
    return trans.augment_image(img)
def img_random_brightness(img):
   #Data Enhancer for Luminance Transform
    brightness = iaa.Multiply((0.3,1.2))
    return brightness.augment_image(img)
def img_gaussian_blur(img):
    #Gaussian Blur Data Augmenter
    gaussian_blur = iaa.GaussianBlur(sigma=(0.1,3))
    return gaussian_blur.augment_image(img)
def img_motion_blur(img):
     #motion Data Augmenter
    motion_blur = iaa.MotionBlur()
    return motion_blur.augment_image(img)
# def img_flip(img,steering_angle):
#     flip = iaa.Fliplr(1.0)
#     return flip.augment_image(img),-steering_angle

## 自定义组合算法， 随机的数据增强。
## path，图片路径。 steering_angle方向盘角度
def random_augment(path,steering_angle):
    image = plt.imread(path)
    if np.random.rand()<0.5:
        image = zoom_img(image)
    if np.random.rand()<0.5:
        image = trans_img(image)
    if np.random.rand()<0.5:
        image = img_random_brightness(image)
    if np.random.rand()<0.5:
        image = img_gaussian_blur(image)
    if np.random.rand()<0.5:
        image = img_motion_blur(image)
#     if np.random.rand()<0.5:
#         image,steering_angle = img_flip(image,steering_angle)   
    return image,steering_angle

In [22]:
import cv2
def train_batch_generator(batch_size):
    while True:
        batch_img = [] # 批量生成的图片
        batch_angle = [] # 每个图片对应一个角度
        for i in range(batch_size):
            ## 从原始的训练集里面随机获取一张照片
            index = np.random.randint(0,len(X_train))
            ## 获取训练集图片的路径
            path = X_train.iloc[index]
            ## 获取训练集图片对应的方向盘角度
            angle = float(y_train.iloc[index])         
            newimg, newangle = random_augment(path,angle)
            ## 对训练数据进行预处理
            newimg = newimg[60:135,:,:]
            newimg = cv2.GaussianBlur(newimg,(3,3),1)
            newimg = cv2.resize(newimg,(200,66))
            newimg = cv2.cvtColor(newimg,cv2.COLOR_RGB2YUV)
            newimg = newimg/255.0
            
            batch_img.append(newimg)
            batch_angle.append(newangle)
        yield np.array(batch_img),np.array(batch_angle)

In [23]:
import cv2
def test_batch_generator(batch_size):
    while True:
        batch_img = [] # 批量生成的图片
        batch_angle = [] # 每个图片对应一个角度
        for i in range(batch_size):
            ## 从原始的训练集里面随机获取一张照片
            index = np.random.randint(0,len(X_test))
            ## 获取训练集图片的路径
            path = X_test.iloc[index]
            ## 获取训练集图片对应的方向盘角度
            angle = float(y_test.iloc[index])         
            newimg, newangle = random_augment(path,angle)
            ## 对训练数据进行预处理
            newimg = newimg[60:135,:,:]
            newimg = cv2.GaussianBlur(newimg,(3,3),1)
            newimg = cv2.resize(newimg,(200,66))
            newimg = cv2.cvtColor(newimg,cv2.COLOR_RGB2YUV)
            newimg = newimg/255.0
            
            batch_img.append(newimg)
            batch_angle.append(newangle)
        yield np.array(batch_img),np.array(batch_angle)

In [24]:
import tensorflow as tf 
# 创建一个顺序模型
model = tf.keras.Sequential()
# normalnation后的第一层
model.add(tf.keras.layers.Conv2D(24,(5,5),strides=(2,2),input_shape=(66,200,3),activation='elu'))
# 第二层卷积
model.add(tf.keras.layers.Conv2D(36,(5,5),strides=(2,2),activation='elu'))
# 第三层卷积
model.add(tf.keras.layers.Conv2D(48,(5,5),strides=(2,2),activation='elu'))
# 第四层卷积
model.add(tf.keras.layers.Conv2D(64,(3,3),activation='elu'))
# 第五层卷积
model.add(tf.keras.layers.Conv2D(64,(3,3),activation='elu'))

# 防止训练过拟合
model.add(tf.keras.layers.Dropout(0.5))

# 全连接打平
model.add(tf.keras.layers.Flatten())

# 全连接的隐藏层
model.add(tf.keras.layers.Dense(1164,activation='elu'))
# 防止训练过拟合
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(100,activation='elu'))
# 防止训练过拟合
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(50,activation='elu'))
# 防止训练过拟合
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(10,activation='elu'))

model.add(tf.keras.layers.Dense(1)) # 方向盘的角度。

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 31, 98, 24)        1824      
                                                                 
 conv2d_11 (Conv2D)          (None, 14, 47, 36)        21636     
                                                                 
 conv2d_12 (Conv2D)          (None, 5, 22, 48)         43248     
                                                                 
 conv2d_13 (Conv2D)          (None, 3, 20, 64)         27712     
                                                                 
 conv2d_14 (Conv2D)          (None, 1, 18, 64)         36928     
                                                                 
 dropout_8 (Dropout)         (None, 1, 18, 64)         0         
                                                                 
 flatten_2 (Flatten)         (None, 1152)             

In [25]:
model.compile(optimizer='adam',loss='mse')
## 使用数据增强器训练的话，api有细微差别
model.fit_generator(train_batch_generator(1000),
                    steps_per_epoch=10,
                    epochs=10,
                   validation_data=test_batch_generator(300),
                    validation_steps=10
                   )

C:\Users\77416\anaconda3\envs\tfenv\lib\site-packages\ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/10
10/10 [==============================] - 107s 12s/step - loss: 2.0814 - val_loss: 0.2581
Epoch 2/10
10/10 [==============================] - 102s 11s/step - loss: 0.6405 - val_loss: 0.0411
Epoch 3/10
10/10 [==============================] - 82s 9s/step - loss: 0.1734 - val_loss: 0.0395
Epoch 4/10
10/10 [==============================] - 81s 9s/step - loss: 0.0924 - val_loss: 0.0378
Epoch 5/10
10/10 [==============================] - 82s 9s/step - loss: 0.0618 - val_loss: 0.0383
Epoch 6/10
10/10 [==============================] - 95s 10s/step - loss: 0.0515 - val_loss: 0.0384
Epoch 7/10
10/10 [==============================] - 82s 9s/step - loss: 0.0470 - val_loss: 0.0383
Epoch 8/10
10/10 [==============================] - 87s 9s/step - loss: 0.0461 - val_loss: 0.0385
Epoch 9/10
10/10 [==============================] - 84s 9s/step - loss: 0.0460 - val_loss: 0.0387
Epoch 10/10
10/10 [==============================] - 84s 9s/step - loss: 0.0444 - val_loss: 0.0364


In [27]:
model.save('004.h5')